In [10]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import math
import time
from collections import Counter
from datetime import date
from datetime import datetime
import matplotlib.pyplot as plt
import re
import csv

In [11]:
driver = webdriver.Chrome(executable_path="C:\Program Files (x86)\Google\Chrome\chromedriver")

In [12]:
def login(usern,passw):
    driver.get('https://www.instagram.com')
    time.sleep(1)
    username = driver.find_element_by_name('username')
    password = driver.find_element_by_name('password')
    username.send_keys(usern)
    password.send_keys(passw)
    driver.find_element_by_tag_name('button').submit()
    try:
        driver.find_element_by_xpath('//div[@class = "piCib"]/div[@class="mt3GC"]/button[last()]').click()
    except NoSuchElementException:
        return
        

def logout():
    driver.find_element_by_xpath('//div[contains(@class,"MWDvN")]//div[@class= "XrOey"][last()]/a').click()
    driver.find_element_by_xpath('//div[@class="AFWDX"]').click()
    driver.find_element_by_xpath('//div[contains(@class,pbNvD)]//button[contains(@class,aOOlW)][last()-1]').click()

def search(tosearch):
    driver.get('https://www.instagram.com/')
    driver.find_element_by_xpath('//div[contains(@class,"LWmhU")]/div').click()
    driver.find_element_by_xpath('//div[contains(@class,"LWmhU")]/input').send_keys(tosearch)
    
def allnames(tosearch):
    search(tosearch)
    ans = []
    for i in  driver.find_elements_by_xpath('//div[@class = "drKGC"]/div[@class="fuqBx"]/a[contains(@class,"yCE8d")]'):
        data = BeautifulSoup(i.get_attribute('innerHTML'),'html.parser')
        if(data.div.div['class'][0] == 'RR-M-'):
            ans.append(data.find_all('div')[0].find_all('div')[1].span.string)
    return ans

driver.get('https://www.instagram.com/')
driver.maximize_window()
a = driver.find_element_by_link_text('Log in')
a.click()
driver.implicitly_wait(10)
driver.refresh()
username = "SAMPLEUSERNAME"
password= "SAMPLEPASSWORD"
login(username,password)
##logout()

In [ ]:

def Top5withMaxFollowers(tosearch):                ##To get 5 handles with max followers in the top 10 handles in search
    search(tosearch)
    ans = {}
    i = 0
    count = 0
    while(count < 10):
        j = driver.find_element_by_xpath('//div[@class = "drKGC"]/div[@class="fuqBx"]/a[contains(@class,"yCE8d")][position() = '+ str(i+1)+']')
        data = BeautifulSoup(j.get_attribute('innerHTML'),'html.parser')
        i+=1
        if(data.div.div['class'][0] == 'RR-M-'):
            j.click()
            count+=1
            s = driver.find_element_by_xpath('//section[@class = "zwlfE"]/ul/li[position() = 2]/a/span').get_attribute('title') ##Gives Followers
            s = s.replace(',',"");
            s = int(s)                             ##To Convert it to Interger                                     
            ans[driver.find_element_by_xpath('//section[@class = "zwlfE"]/div/h1').text] = s
            search(tosearch)
        time.sleep(1)
    ans = Counter(ans).most_common(5)              ##To get top 5 in the answers
    return ans

ans = Top5withMaxFollowers('food')
print("Pages with max followers are:")
for i in ans:
    print(i[0]," ",i[1])
    
ansfinal = {}
postin3days = []
names = []
for q in ans:
    search(q[0])
    i = driver.find_element_by_xpath('//div[@class = "drKGC"]/div[@class="fuqBx"]/a[contains(@class,"yCE8d")]')
    i.click()        
    count = 0
    li = driver.find_elements_by_xpath('//div[contains(@class,"_2z6nI")]//div[contains(@class,"v1Nh3")]')
    while(1):
        li[count].click()
        postdate = driver.find_element_by_xpath('//a[@class= "c-Yi7"]/time').get_attribute('datetime')
        postdate = postdate.split('T')[0]
        postdate = [int(i) for i in postdate.split('-')]
        postdate = datetime(postdate[0],postdate[1],postdate[2])        ##Converting Date(string format) to datetime format
        if((datetime.today()-postdate).days <=3 and len(li) > count):   ##Continue if The posts are within 3 days and posts are left to see.
            count+=1
        else:                                                           ##If no post is left or they are more then 3 days ago.
            break
        time.sleep(2)
        driver.find_element_by_xpath('//button[@class = "ckWGn"]').click()##Closing the post
        if(count//12 >  (count-1)//12 or len(li) <= count):              ##To load more posts we need to scroll
            driver.execute_script('window.scrollTo(0,arguments[0])',driver.execute_script('return document.body.scrollHeight'))
            li = driver.find_elements_by_xpath('//div[contains(@class,"_2z6nI")]//div[contains(@class,"v1Nh3")]')
            driver.execute_script('window.scrollTo(0,0)')
            time.sleep(3)
        time.sleep(1)
    ansfinal[q[0]] = count
    postin3days.append(count)
    names.append(q[0])
print("There number of posts ,posted in 3 days for corresponding pages are:")
for i in ansfinal.keys():
    print(i," ",ansfinal[i])

In [ ]:
plt.rcParams["figure.figsize"] = [15,5] 
xpos = np.arange(0,5,1)
plt.bar(xpos,postin3days,width = 0.3,color = "purple")
plt.xticks(xpos,names)
plt.title("Handle names vs Their posts in 3 days")
for i in range(5):
    plt.text(i-0.03,postin3days[i]+0.3,postin3days[i])
plt.show()

In [ ]:
allWords = {}                ## To store all the words in the post
hashtags = {}                ## To Store the hashtags 
for q in names:              ## For the handles found earlier
    search(q)
    time.sleep(1)
    i = driver.find_element_by_xpath('//div[@class = "drKGC"]/div[@class="fuqBx"]/a[contains(@class,"yCE8d")]')
    i.click()        
    count = 0
    li = driver.find_elements_by_xpath('//div[contains(@class,"_2z6nI")]//div[contains(@class,"v1Nh3")]')
    while(count < 10):                                    ##Scraping the top 10 posts
        ##If total posts are less than 10
        if(count == len(li)):                              
            break
        
        li[count].click()
        document = driver.find_element_by_xpath('//div[@class = "EtaWk"]//div[@class = "C4VMK"]/span').text
        # to remove all the special character
        document = re.sub(r'\W', ' ', document)

        # to convert multiple spaces into single spaces
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removed prefixed b
        document = re.sub(r'^b\s+', '', document)

        # remove integers
        document = re.sub(r'\d+', '', document)

        # convert all words into lowerletter
        document = document.lower()
        document = document.split()
        ## Storing all the words
        for i in document:
            allWords[i] = allWords.get(i,0)+1
            
        ##Getting all the hashtags
        temp = driver.find_elements_by_xpath('//div[@class = "P9YgZ"]/div/div[position() = 2]/span/a')
        for i in temp:
            ## If it is a hashtag only
            if(i.text[0] == '#'): 
                hashtags[i.text[1:]] = hashtags.get(i.text[1:],0) +1
        driver.find_element_by_xpath('//button[@class = "ckWGn"]').click()
        time.sleep(1)
        count+=1
        
##Creating a csv file
with open('allwords.csv', 'w',encoding = "utf-8") as f:
    for key in allWords.keys():
        f.write("%s,%s\n"%(key,allWords[key]))

In [ ]:
hashtags = {k: v for k, v in sorted(hashtags.items(), key=lambda item: item[1],reverse = True)}
top5hashtags = []
top5sfreq = []
count = 0

##Custom function to pass in the autopct for pie chart
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct


##Printing all the hashtags
print("most frequent hashtags are:")
for i in hashtags:
    if(count < 5):
        top5hashtags.append(i);
        top5sfreq.append(hashtags[i])
        count+=1
    print(i," " , hashtags[i])
plt.pie(top5sfreq,labels = top5hashtags,autopct = make_autopct(top5sfreq))
plt.title("Top Hashtags and their Freq")
plt.show()

In [ ]:
ratio = []                 ##To store correponding ratios
for q in names:
    search(q)
    time.sleep(1)
    i = driver.find_element_by_xpath('//div[@class = "drKGC"]/div[@class="fuqBx"]/a[contains(@class,"yCE8d")]')
    i.click()        
    count = 0   ##To store total posted encountered(incase total posts are less than 10)
    sumt = 0    ##To store total likes on the posts
    li = driver.find_elements_by_xpath('//div[contains(@class,"_2z6nI")]//div[contains(@class,"v1Nh3")]')
    while(count < 10):
        ## To check If total posted are less than 10 
        if(count == len(li)):
            break
        li[count].click()
        t = driver.find_element_by_xpath('//section[contains(@class,"EDfFK")]/div').get_attribute('class')
        ##If the views is greater than likes
        if (t == "HbPOm _9Ytll"):
            driver.find_element_by_xpath('//div[contains(@class,"HbPOm")]/span/span').click()
            s = driver.find_element_by_xpath('//div[@class = "vJRqr"]/span').text
            s = s.replace(",","")
            s = int(s)
            sumt+=s
        else:
            s = driver.find_element_by_xpath('//div[@class = "Nm9Fw"]//span').text
            s = s.replace(',',"")
            s = int(s)
            sumt+=s
            
        temp = driver.find_elements_by_xpath('//div[@class = "P9YgZ"]/div/div[position() = 2]/span/a')
        driver.find_element_by_xpath('//button[@class = "ckWGn"]').click()
        time.sleep(1)
        count+=1
    ##To Get the number of Followers
    s = driver.find_element_by_xpath('//ul[@class = "k9GMp "]//li[@class = "Y8-fY "][position() = 2]/a/span').get_attribute('title')
    s = s.replace(',',"")
    s = int(s)
    ratio.append(round((sumt/count)/s,6))
    
for i in range(5):
    print(names[i]," ",ratio[i])

In [ ]:
xpos = np.arange(1,6,1)
plt.rcParams["figure.figsize"] = [15,3]
plt.bar(xpos,ratio,width = 0.5,color = "purple")
plt.xticks(xpos,names)
plt.xlabel("Handles", fontweight="bold" )
plt.ylabel("Ratio", fontweight="bold")
plt.title("Average Followers:Like ratio")

for i in range(5):
    plt.text(i+1-0.15,ratio[i]+0.0007,ratio[i])
plt.show()